# Exploratory Data Analysis

In [ ]:
import pandas
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [14]:
# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "/home/jovyan/work/airflow-hackernews-github-365ae261a883.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "airflow-hackernews-github"

process_date = "20200101"
process_date_dash = "2020-01-01"

## Githubarchive dataset

### What type of event is most pupolar

In [15]:
#What type of event is most popular
sql = f"""
    SELECT 
        type, COUNT(*) AS count_
    
    FROM `githubarchive.day.{process_date}`
    GROUP BY 1
    ORDER BY 2 DESC
"""

print(sql)


    SELECT 
        type, COUNT(*) AS count_
    
    FROM `githubarchive.day.20200101` 
    GROUP BY 1
    ORDER BY 2 DESC



In [16]:
df = pandas_gbq.read_gbq(sql, dialect='standard')
print(df)

Downloading: 100%|██████████| 14/14 [00:01<00:00, 12.40rows/s]

                             type  count_
0                       PushEvent  570350
1                     CreateEvent  145556
2                      WatchEvent   76352
3                PullRequestEvent   72025
4               IssueCommentEvent   49779
5                     DeleteEvent   30266
6                     IssuesEvent   27008
7                       ForkEvent   24715
8   PullRequestReviewCommentEvent    8609
9                     GollumEvent    4434
10                   ReleaseEvent    3920
11                    PublicEvent    2574
12             CommitCommentEvent    1984
13                    MemberEvent    1508


### Top 10 repos with the most comments in their issues

In [24]:
sql = f"""
    SELECT 
        repo.name,
        COUNT(*) AS count_
    FROM `githubarchive.day.{process_date}`
    WHERE type = 'IssueCommentEvent'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10
"""
print(sql)


    SELECT 
        repo.name,
        COUNT(*) AS count_
    FROM `githubarchive.day.20200101`
    WHERE type = 'IssueCommentEvent'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10



In [25]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.64rows/s]

                                        name  count_
0               angband-import/angband-trac2    2915
1                               hankcs/HanLP     563
2             google-test/signcla-probe-repo     384
3    test-organization-kkjeer/bot-validation     196
4          test-organization-kkjeer/app-test     195
5                          dotnet/docs.ja-jp     168
6                             rust-lang/rust     164
7                      kubernetes/kubernetes     161
8                          pandas-dev/pandas     158
9  bcbi-test/automerge-integration-test-repo     140


### Top 10 repos by watch and fork events

In [28]:
sql = f"""
    SELECT
        repo.name,
        SUM(IF(type='WatchEvent', 1, 0)) AS numOfWatchEvents,
        SUM(IF(type='ForkEvent', 1, 0)) AS numOfForkEvents,
        COUNT(*) AS count_
    FROM `githubarchive.day.{process_date}`
    WHERE type IN ('WatchEvent', 'ForkEvent')
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10
"""
print(sql)


    SELECT
        repo.name,
        SUM(IF(type='WatchEvent', 1, 0)) AS numOfWatchEvents,
        SUM(IF(type='ForkEvent', 1, 0)) AS numOfForkEvents,
        COUNT(*) AS count_
    FROM `githubarchive.day.20200101`
    WHERE type IN ('WatchEvent', 'ForkEvent')
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10



In [29]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.16rows/s]

                                  name  numOfWatchEvents  numOfForkEvents  \
0               victorqribeiro/isocity               615               35   
1                      sw-yx/spark-joy               266                9   
2                     imsnif/bandwhich               265               13   
3             socialpoint-labs/sheetfu               236               10   
4                   pingcap/chaos-mesh               220                6   
5                         pakastin/car               219               15   
6  feelschaotic/AndroidKnowledgeSystem               215               32   
7                 testerSunshine/12306               210               83   
8                     mayeaux/nodetube               195               13   
9                 synesthesiam/rhasspy               175                3   

   count_  
0     650  
1     275  
2     278  
3     246  
4     226  
5     234  
6     247  
7     293  
8     208  
9     178  


## Hackernews Dataset

### Top 10 domain ranked by average score

In [34]:
sql = """
    SELECT 
        REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
        AVG(score) AS average_score,
        COUNT(*) AS count_
    FROM `bigquery-public-data.hacker_news.full`
    WHERE url != ''
    GROUP BY 1
    ORDER BY 3 DESC, 2 DESC
    LIMIT 10
"""
print(sql)


    SELECT 
        REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
        AVG(score) AS average_score,
        COUNT(*) AS count_
    FROM `bigquery-public-data.hacker_news.full`
    WHERE url != ''
    GROUP BY 1
    ORDER BY 3 DESC, 2 DESC
    LIMIT 10



In [35]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.94rows/s]

                domain  average_score  count_
0           medium.com       7.707072   94542
1           github.com      16.416370   92240
2      www.youtube.com       4.578248   67094
3      www.nytimes.com      20.502744   47008
4       techcrunch.com      17.591803   41017
5      arstechnica.com      14.925032   27385
6     en.wikipedia.org       9.789674   20763
7  www.theguardian.com      17.142886   20016
8        www.wired.com      13.698954   19698
9    www.bloomberg.com      24.790660   19657


### Top 10 domain ranked by number of scores greater than 40

In [39]:
sql = """
    SELECT 
        REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
        SUM(IF(score>40, 1, 0)) num>40,
        COUNT(*) AS count_
    FROM `bigquery-public-data.hacker_news.full`
    WHERE url != ''
    GROUP BY 1
    ORDER BY 2 DESC, 3 DESC
    LIMIT 10
"""
print(sql)


    SELECT 
        REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
        SUM(IF(score>40, 1, 0)),
        COUNT(*) AS count_
    FROM `bigquery-public-data.hacker_news.full`
    WHERE url != ''
    GROUP BY 1
    ORDER BY 2 DESC, 3 DESC
    LIMIT 10



In [40]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.08rows/s]

                   domain   f0_  count_
0              github.com  8274   92240
1         www.nytimes.com  5617   47008
2          techcrunch.com  4067   41017
3              medium.com  2881   94542
4       www.bloomberg.com  2739   19657
5         arstechnica.com  2311   27385
6     www.theguardian.com  1867   20016
7           www.wired.com  1597   19698
8  www.washingtonpost.com  1541   13609
9             www.bbc.com  1537   13310


### Top 10 Hackernews stories from Github by score

In [48]:
sql = """
    SELECT 
        `by`,
        id AS story_id,
        REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") AS url,
        SUM(score) AS score
    FROM `bigquery-public-data.hacker_news.stories`
    WHERE url LIKE '%https://github.com%'
        AND url NOT LIKE '%github.com/blog/%'
    GROUP BY 1,2,3
    ORDER BY 4 DESC
    LIMIT 10
"""
print(sql)


    SELECT 
        `by`,
        id AS story_id,
        REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") AS url,
        SUM(score) AS score
    FROM `bigquery-public-data.hacker_news.stories`
    WHERE url LIKE '%https://github.com%'
        AND url NOT LIKE '%github.com/blog/%'
    GROUP BY 1,2,3
    ORDER BY 4 DESC
    LIMIT 10



In [49]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.09rows/s]

            by  story_id                                                url  \
0  realfuncode  10101469     https://github.com/shadowsocks/shadowsocks-iOS   
1   hannahmitt  10204018           https://github.com/HannahMitt/HomeMirror   
2    peteretep   2661209                https://github.com/MrMEEE/bumblebee   
3    peterhunt   9271246           https://github.com/facebook/react-native   
4    timmorgan   8085213                https://github.com/churchio/onebody   
5    codeulike   6048761                https://github.com/mame/quine-relay   
6      tarruda   7278214                   https://github.com/neovim/neovim   
7       chadrs   3287272  https://github.com/chadselph/OOptOut-Chrome-Ex...   
8     onestone   9542267                      https://github.com/iojs/io.js   
9    DaGardner   9196218                  https://github.com/mackyle/sqlite   

   score  
0   1543  
1   1172  
2   1123  
3   1039  
4   1020  
5    983  
6    879  
7    820  
8    794  
9    792  


## Example of Final Table: GitHub on Hacker News Trends of 2020-01-01

In [60]:
sql = """

WITH github_activity AS (
    SELECT 
        repo.name AS repo,
        CONCAT('https://github.com/', repo.name) AS url,
        SUM(IF(type='WatchEvent', 1, 0)) AS numOfWatchEvents,
        SUM(IF(type='ForkEvent', 1, 0)) AS numOfForkEvents,
        COUNT(*) AS count_
    FROM `githubarchive.day.20200101`
    WHERE type IN ('WatchEvent', 'ForkEvent')
    GROUP BY 1,2
),
hacker_news AS (
    SELECT 
        EXTRACT(DATE FROM timestamp) AS date,
        `by` AS submitter,
        id AS story_id,
        REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") AS url,
        SUM(score) AS score
    FROM `bigquery-public-data.hacker_news.full`
    WHERE
        type='story'
        AND EXTRACT(DATE FROM timestamp)='2020-01-01'
        AND url LIKE '%https://github.com%'
        AND url NOT LIKE '%github.com/blog/%'
    GROUP BY 1,2,3,4
)

SELECT 
    a.date AS date,
    b.repo AS github_repo,
    a.score AS score,
    a.story_id AS story_id,
    b.numOfWatchEvents AS numOfWatchEvents,
    b.numOfForkEvents AS numOfForkEvents
FROM hacker_news AS a
LEFT JOIN github_activity AS b
ON a.url = b.url
ORDER BY score DESC
LIMIT 10

"""

print(sql)



WITH github_activity AS (
    SELECT 
        repo.name AS repo,
        CONCAT('https://github.com/', repo.name) AS url,
        SUM(IF(type='WatchEvent', 1, 0)) AS numOfWatchEvents,
        SUM(IF(type='ForkEvent', 1, 0)) AS numOfForkEvents,
        COUNT(*) AS count_
    FROM `githubarchive.day.20200101`
    WHERE type IN ('WatchEvent', 'ForkEvent')
    GROUP BY 1,2
),
hacker_news AS (
    SELECT 
        EXTRACT(DATE FROM timestamp) AS date,
        `by` AS submitter,
        id AS story_id,
        REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") AS url,
        SUM(score) AS score
    FROM `bigquery-public-data.hacker_news.full`
    WHERE
        type='story'
        AND EXTRACT(DATE FROM timestamp)='2020-01-01'
        AND url LIKE '%https://github.com%'
        AND url NOT LIKE '%github.com/blog/%'
    GROUP BY 1,2,3,4
)

SELECT 
    a.date AS date,
    b.repo AS github_repo,
    a.score AS score,
    a.story_id AS story_id,
    b.numOfWatchEvents AS numOfWatchEvent

In [61]:
df = pandas_gbq.read_gbq(sql, dialect = 'standard')
print(df)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.03rows/s]

        date                github_repo  score  story_id  numOfWatchEvents  \
0 2020-01-01   socialpoint-labs/sheetfu    224  21928325             236.0   
1 2020-01-01           mayeaux/nodetube    151  21926666             195.0   
2 2020-01-01             falcon78/Vusic    102  21928211             107.0   
3 2020-01-01         abhiTronix/vidgear     89  21926922             148.0   
4 2020-01-01            chaskiq/chaskiq     32  21929676               9.0   
5 2020-01-01            wassuphq/wassup     13  21931615               2.0   
6 2020-01-01  alexellis/faas-containerd     10  21928642               1.0   
7 2020-01-01             kyai/redis-cui      6  21926151               1.0   
8 2020-01-01             spypunk/sponge      6  21926740               4.0   
9 2020-01-01                       None      3  21927644               NaN   

   numOfForkEvents  
0             10.0  
1             13.0  
2             10.0  
3              3.0  
4              1.0  
5              